In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from graph_utils import graph_gen, find_alpha_lower_bound
from algorithms import IPLUX, UDC

# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# from algorithms import B_DPP, Falsone, C_SP_SG, DPD_TV

num_node = 5
num_edge = 7

np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

Graph with 5 nodes and 7 edges
(5, 5)


In [5]:
from problem import Synthetic

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3 # dimension of x_i's
parameters['m'] = 1 # number jof inequality constraints
parameters['p'] = 5 # number of equality constraints

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

generating a Synthetic problem: N=5, d=3, m=1, p=5
Q: (5, 3), P: (5, 3, 3)
A: (5, 5, 3)
a: (5, 3), c: (5,)
aa: (5, 3), cc: (5,)
x* (5, 3), f* 5.484875848760243e-10
generated problem saved in data/problem/Synthetic/N5

loading a Synthetic problem, N=5
problem loaded:
Q: (5, 3), P: (5, 3, 3)
A: (5, 5, 3)
a: (5, 3), c: (5,)
aa: (5, 3), ca: (5,)
x_star (5, 3)
opt_val 5.484875848760243e-10
x* [[ 4.15925155e-10  4.83162016e-10 -7.59907459e-11]
 [ 8.50751131e-10 -2.35516155e-10 -9.33097279e-12]
 [ 4.04377111e-10 -1.25635409e-09 -9.34978611e-11]
 [-9.37742232e-10 -1.40009881e-11 -1.51506828e-11]
 [ 2.71163230e-11  5.99188639e-10 -3.77198993e-11]]
||x*|| 2.0452886690946836e-09



In [13]:
alg = UDC(prob, network, rho=1, alpha=1, param_setting='proximal_tracking', verbose=False)

for i in range(100):
    alg.step()

UDC setting: proximal_tracking
self.prob minimize quad_over_lin(param2000 @ var1999, 1.0) + param2001 @ var1999 + norm1(var1999) + 0.5 @ QuadForm(var1999 + -param2002, [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]) + quad_over_lin(maximum(param2005 + quad_over_lin(var1999 + -param2003, 1.0) + -param2004, 0.0), 1.0)
subject to QuadForm(var1999 + -param2010, [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]) <= param2011[0]
reset
UDC_pt alpha 1 rho 1, iter 0, obj err: 5.48e-10, cons vio: 0.00e+00
time 0.01, saved



In [ ]:
MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
rho_choice = [0.1, 0.5, 2, 5]
alpha_choice = [alpha_lower+6]
rho_choice = [2]


np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
        for i in range(MAX_ITER):
            alg.step()

self.prob minimize param7916 @ var7915 + norm1(var7915) + 5.0 @ QuadForm(var7915 + -param7917, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + 0.25 @ quad_over_lin(param7918 @ var7915, 1.0) + param7919 @ var7915 + param7920[0] @ QuadForm(var7915, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + Promote(-2.0, (3,)) @ param7921 @ var7915
subject to QuadForm(var7915 + -param7923, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param7924[0]
reset
IPLUX alpha 10.0 rho 2, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.01, saved

IPLUX alpha 10.0 rho 2, iter 100, obj err: 8.31e-03, cons vio: 6.39e-02
time 6.17, saved

IPLUX alpha 10.0 rho 2, iter 200, obj err: 4.39e-03, cons vio: 3.45e-02
time 12.73, saved

IPLUX alpha 10.0 rho 2, iter 300, obj err: 2.84e-03, cons vio: 1.39e-02
time 19.39, saved

IPLUX alpha 10.0 rho 2, iter 400, obj err: 7.76e-04, cons vio: 4.79e-03
time 25.65, saved

IPLUX alpha 10.0 rho 2, iter 500, obj err: 2.58e-04, cons vio: 1.61e-03


In [19]:
# =========================================================================== |
# ---------------------------------- Plot ----------------------------------- |
# =========================================================================== |

from plot_utils import MyFigure


# alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
# rho_choice = [0.1, 0.5, 2, 5, 10]
# rho_choice = [0.05, 0.1, 0.5, 2, 5]
MAX_ITER = 2000
alpha_choice = [alpha_lower+6]
rho_choice = [2]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
# marker = [".", "o", "^", "s", "p", "P", "*"]
linestyle = ['-', '--', '-.', ':', '']



# ================================ objective error============================
obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel=r'$|\sum_{i=1}^N f_i(x_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)|$',
                            yscale='log')
# obj_err_figure.add_line("IPLUX", alg.obj_err_log)
for rho in rho_choice:
    for alpha in alpha_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'IPLUX_a{alpha}_r{rho}'
        filename = f'log/N{num_node}/{prefix}_oe.txt'
        obj_err_figure.add_line_file(prefix, filename, style=c+ls)
obj_err_figure.paint(MAX_ITER=MAX_ITER)


# ================================= constraint violation ====================
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='constraint violation',
                            yscale='log')
for rho in rho_choice:
    for alpha in alpha_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'IPLUX_a{alpha}_r{rho}'
        filename = f'log/N{num_node}/{prefix}_cv.txt'
        cons_vio_figure.add_line_file(prefix, filename, style=c+ls)
cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


# ================================= x distance ==============================
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='x distance',
                            yscale='log')
for rho in rho_choice:
    for alpha in alpha_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'IPLUX_a{alpha}_r{rho}'
        filename = f'log/N{num_node}/{prefix}_xd.txt'
        x_dis_figure.add_line_file(prefix, filename, style=c+ls)
x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


In [ ]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


In [88]:
# print(alg.W)
# print(alg.H)
np.linalg.norm(prob.x_star)

1.5972494728972766